# Criando Scorecard
---

In [1]:
# importanto bibliotecas
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import metrics

from funcoes.funcoes import *

import joblib

import warnings
warnings.filterwarnings("ignore")

In [2]:
# importando modelo de probabilidade de inadimplência
modelo_pd = joblib.load('modelos/modelo_pd_v1.sav')

In [3]:
# importando dados
dados_WoE_VI   = pd.read_csv('dados/dados_WoE_VI_final.csv')

In [4]:
dados_WoE_VI.shape

(877648, 79)

In [5]:
# separando dados
X = dados_WoE_VI.drop(['target', 'funded_amnt', 'total_rec_prncp', 'loan_status', 'recoveries'], axis = 1).values
y = dados_WoE_VI[['target']].values

In [6]:
# tratando o desbalanceamento das classes da variável target
smote = SMOTE(random_state = 42)
X_smote, y_smote = smote.fit_resample(X, y)

In [7]:
# separando dados de teste e treino
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size = 0.3, random_state = 42)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1068943, 77)
(458119, 77)
(1068943,)
(458119,)


In [9]:
reg = LogisticRegression_with_p_values()
reg.fit(X_train, y_train)

In [10]:
nomes_features = dados_WoE_VI.drop(['target', 'funded_amnt', 'total_rec_prncp', 'loan_status', 'recoveries'], axis = 1).columns.values

In [11]:
tabela_sumario = pd.DataFrame(columns = ['nome_feature'], data = nomes_features)
tabela_sumario['coeficiente'] = np.transpose(reg.coef_)
tabela_sumario.index = tabela_sumario.index + 1
tabela_sumario.loc[0] = ['intercept', reg.intercept_[0]]
tabela_sumario = tabela_sumario.sort_index()

p_values = reg.p_values
p_values = np.append(np.nan,np.array(p_values))
tabela_sumario['p_valor'] = p_values
tabela_sumario

,nome_feature,coeficiente,p_valor
0,intercept,-10.076345,NaN
1,purpose:credit_card,2.258435,0.0
2,purpose:debt_consolidation,1.998950,0.0
3,purpose:home_improvement,2.635993,0.0
4,home_ownership:OWN,0.845471,0.0
...,...,...,...
73,total_rev_hi_lim:10000-20000,1.569981,0.0
74,total_rev_hi_lim:20000-40000,1.538325,0.0
75,total_rev_hi_lim:40000-60000,1.761187,0.0
76,total_rev_hi_lim:60000-80000,2.067609,0.0


Agora foi criado um score calculado com base nos coeficientes, em que coeficientes maiores vão corresponder consequentemente a melhores mutuários.

In [12]:
# criando um dataframe
df_ref_categorias = pd.DataFrame()

# concatenando o datafram com a tabela sumario
df_scorecard = pd.concat([tabela_sumario, df_ref_categorias])
df_scorecard = df_scorecard.reset_index()

df_scorecard['nome_original_feature'] = df_scorecard['nome_feature'].str.split(':').str[0]
df_scorecard

,index,nome_feature,coeficiente,p_valor,nome_original_feature
0,0,intercept,-10.076345,NaN,intercept
1,1,purpose:credit_card,2.258435,0.0,purpose
2,2,purpose:debt_consolidation,1.998950,0.0,purpose
3,3,purpose:home_improvement,2.635993,0.0,purpose
4,4,home_ownership:OWN,0.845471,0.0,home_ownership
...,...,...,...,...,...
73,73,total_rev_hi_lim:10000-20000,1.569981,0.0,total_rev_hi_lim
74,74,total_rev_hi_lim:20000-40000,1.538325,0.0,total_rev_hi_lim
75,75,total_rev_hi_lim:40000-60000,1.761187,0.0,total_rev_hi_lim
76,76,total_rev_hi_lim:60000-80000,2.067609,0.0,total_rev_hi_lim


Os valores de score do modelo vão de 0 a 1000, padrão utilizado por muitas empresas.

In [13]:
min_score = 0
max_score = 1000

In [14]:
# agrupando dados e calculando o coeficiente mínimo de cada variável
df_scorecard.groupby('nome_original_feature')['coeficiente'].min()

nome_original_feature
annual_inc              1.446691
dti                    -1.829630
funded_amnt             2.277283
grade                  -1.816148
home_ownership          0.267596
initial_list_status     0.302850
inq_last_6mths          0.101726
int_rate                0.612532
intercept             -10.076345
max_bal_bc              0.076071
mths_since_rcnt_il      0.579543
purpose                 1.998950
revol_util              1.023415
term                    0.420562
tot_cur_bal             1.048717
total_bal_il            0.776545
total_rev_hi_lim        1.538325
verification_status     0.592742
Name: coeficiente, dtype: float64

In [15]:
# soma dos coeficientes mínimos
min_sum_coef = df_scorecard.groupby('nome_original_feature')['coeficiente'].min().sum()
min_sum_coef

-0.6585753565881639

In [16]:
# agrupando dados e calculando o coeficiente máximos de cada variável
df_scorecard.groupby('nome_original_feature')['coeficiente'].max()

nome_original_feature
annual_inc              2.386512
dti                    -0.007375
funded_amnt             3.804464
grade                  -0.527742
home_ownership          0.845471
initial_list_status     0.302850
inq_last_6mths          0.513394
int_rate                1.125901
intercept             -10.076345
max_bal_bc              2.797610
mths_since_rcnt_il      1.977493
purpose                 2.863988
revol_util              1.317903
term                    0.420562
tot_cur_bal             1.808018
total_bal_il            1.469801
total_rev_hi_lim        2.358835
verification_status     0.989579
Name: coeficiente, dtype: float64

In [17]:
# soma dos coeficientes máximos
max_sum_coef = df_scorecard.groupby('nome_original_feature')['coeficiente'].max().sum()
max_sum_coef

14.370918951369125

Reescalando coeficientes de cada variável para um score correspondente.

In [18]:
# multiplicação do o valor da coluna coeficientes pela razão das diferenças entre
# pontuação máxima e pontuação mínima e soma máxima e soma mínima dos coeficientes
df_scorecard['calculo_score'] = df_scorecard['coeficiente'] * (max_score - min_score) / (max_sum_coef - min_sum_coef)
df_scorecard

,index,nome_feature,coeficiente,p_valor,nome_original_feature,calculo_score
0,0,intercept,-10.076345,NaN,intercept,-670.438063
1,1,purpose:credit_card,2.258435,0.0,purpose,150.266860
2,2,purpose:debt_consolidation,1.998950,0.0,purpose,133.001802
3,3,purpose:home_improvement,2.635993,0.0,purpose,175.387996
4,4,home_ownership:OWN,0.845471,0.0,home_ownership,56.254144
...,...,...,...,...,...,...
73,73,total_rev_hi_lim:10000-20000,1.569981,0.0,total_rev_hi_lim,104.460003
74,74,total_rev_hi_lim:20000-40000,1.538325,0.0,total_rev_hi_lim,102.353714
75,75,total_rev_hi_lim:40000-60000,1.761187,0.0,total_rev_hi_lim,117.182070
76,76,total_rev_hi_lim:60000-80000,2.067609,0.0,total_rev_hi_lim,137.570068


In [19]:
# divide-se a diferença do valor da coluna coeficientes e a soma mínima dos coeficientes pela
# diferença da soma máxima e a soma mínima dos coeficientes
# multiplica-se pela diferença entre a pontuação máxima e a pontuação mínima
# e adiciona-se a pontuação mínima.
df_scorecard['calculo_score'][0] = ((df_scorecard['coeficiente'][0] - min_sum_coef) / (max_sum_coef - min_sum_coef)) * (max_score - min_score) + min_score
df_scorecard

,index,nome_feature,coeficiente,p_valor,nome_original_feature,calculo_score
0,0,intercept,-10.076345,NaN,intercept,-626.619200
1,1,purpose:credit_card,2.258435,0.0,purpose,150.266860
2,2,purpose:debt_consolidation,1.998950,0.0,purpose,133.001802
3,3,purpose:home_improvement,2.635993,0.0,purpose,175.387996
4,4,home_ownership:OWN,0.845471,0.0,home_ownership,56.254144
...,...,...,...,...,...,...
73,73,total_rev_hi_lim:10000-20000,1.569981,0.0,total_rev_hi_lim,104.460003
74,74,total_rev_hi_lim:20000-40000,1.538325,0.0,total_rev_hi_lim,102.353714
75,75,total_rev_hi_lim:40000-60000,1.761187,0.0,total_rev_hi_lim,117.182070
76,76,total_rev_hi_lim:60000-80000,2.067609,0.0,total_rev_hi_lim,137.570068


In [20]:
# arrendondando o valor do score
df_scorecard['score_preliminar'] = df_scorecard['calculo_score'].round()
df_scorecard

,index,nome_feature,coeficiente,p_valor,nome_original_feature,calculo_score,score_preliminar
0,0,intercept,-10.076345,NaN,intercept,-626.619200,-627.0
1,1,purpose:credit_card,2.258435,0.0,purpose,150.266860,150.0
2,2,purpose:debt_consolidation,1.998950,0.0,purpose,133.001802,133.0
3,3,purpose:home_improvement,2.635993,0.0,purpose,175.387996,175.0
4,4,home_ownership:OWN,0.845471,0.0,home_ownership,56.254144,56.0
...,...,...,...,...,...,...,...
73,73,total_rev_hi_lim:10000-20000,1.569981,0.0,total_rev_hi_lim,104.460003,104.0
74,74,total_rev_hi_lim:20000-40000,1.538325,0.0,total_rev_hi_lim,102.353714,102.0
75,75,total_rev_hi_lim:40000-60000,1.761187,0.0,total_rev_hi_lim,117.182070,117.0
76,76,total_rev_hi_lim:60000-80000,2.067609,0.0,total_rev_hi_lim,137.570068,138.0


In [21]:
# soma de todos os valores mínimos dos valores da coluna coeficiente
min_sum_score_prel = df_scorecard.groupby('nome_original_feature')['score_preliminar'].min().sum()
min_sum_score_prel

0.0

In [22]:
# soma de todos os valores máximos dos valores da coluna coeficiente
# score não está dentro da faixa escolhida
max_sum_score_prel = df_scorecard.groupby('nome_original_feature')['score_preliminar'].max().sum()
max_sum_score_prel

1001.0

In [23]:
df_scorecard['diferenca'] = df_scorecard['score_preliminar'] - df_scorecard['calculo_score']
df_scorecard

,index,nome_feature,coeficiente,p_valor,nome_original_feature,calculo_score,score_preliminar,diferenca
0,0,intercept,-10.076345,NaN,intercept,-626.619200,-627.0,-0.380800
1,1,purpose:credit_card,2.258435,0.0,purpose,150.266860,150.0,-0.266860
2,2,purpose:debt_consolidation,1.998950,0.0,purpose,133.001802,133.0,-0.001802
3,3,purpose:home_improvement,2.635993,0.0,purpose,175.387996,175.0,-0.387996
4,4,home_ownership:OWN,0.845471,0.0,home_ownership,56.254144,56.0,-0.254144
...,...,...,...,...,...,...,...,...
73,73,total_rev_hi_lim:10000-20000,1.569981,0.0,total_rev_hi_lim,104.460003,104.0,-0.460003
74,74,total_rev_hi_lim:20000-40000,1.538325,0.0,total_rev_hi_lim,102.353714,102.0,-0.353714
75,75,total_rev_hi_lim:40000-60000,1.761187,0.0,total_rev_hi_lim,117.182070,117.0,-0.182070
76,76,total_rev_hi_lim:60000-80000,2.067609,0.0,total_rev_hi_lim,137.570068,138.0,0.429932


In [24]:
# desfazendo um dos arredondamentos para que os valores de score fiquem entre o máximo e mínimo selecionados
df_scorecard['score_final'] = df_scorecard['score_preliminar']
df_scorecard['score_final'][77] = 156
df_scorecard

,index,nome_feature,coeficiente,p_valor,nome_original_feature,calculo_score,score_preliminar,diferenca,score_final
0,0,intercept,-10.076345,NaN,intercept,-626.619200,-627.0,-0.380800,-627.0
1,1,purpose:credit_card,2.258435,0.0,purpose,150.266860,150.0,-0.266860,150.0
2,2,purpose:debt_consolidation,1.998950,0.0,purpose,133.001802,133.0,-0.001802,133.0
3,3,purpose:home_improvement,2.635993,0.0,purpose,175.387996,175.0,-0.387996,175.0
4,4,home_ownership:OWN,0.845471,0.0,home_ownership,56.254144,56.0,-0.254144,56.0
...,...,...,...,...,...,...,...,...,...
73,73,total_rev_hi_lim:10000-20000,1.569981,0.0,total_rev_hi_lim,104.460003,104.0,-0.460003,104.0
74,74,total_rev_hi_lim:20000-40000,1.538325,0.0,total_rev_hi_lim,102.353714,102.0,-0.353714,102.0
75,75,total_rev_hi_lim:40000-60000,1.761187,0.0,total_rev_hi_lim,117.182070,117.0,-0.182070,117.0
76,76,total_rev_hi_lim:60000-80000,2.067609,0.0,total_rev_hi_lim,137.570068,138.0,0.429932,138.0


Agora o scorecard está correto, dentro dos valores escolhidos.

In [25]:
min_sum_score_prel = df_scorecard.groupby('nome_original_feature')['score_final'].min().sum()
min_sum_score_prel

0.0

In [26]:
max_sum_score_prel = df_scorecard.groupby('nome_original_feature')['score_final'].max().sum()
max_sum_score_prel

1000.0

### Calculando score de crédito

Aqui foi calculado o score de cada registro do dataset de teste criado abaixo. 

In [27]:
# criando um dataframe com os dados de teste
df_features = pd.DataFrame(columns = nomes_features, data = X_test)

# inserindo nova coluna no dataset
df_features.insert(0, 'intercept', 1)

In [28]:
scorecard_scores = df_scorecard['score_final']

In [29]:
df_features.shape

(458119, 78)

In [30]:
scorecard_scores.shape

(78,)

In [31]:
scorecard_scores = scorecard_scores.values.reshape(78, 1)

In [32]:
# multiplicando os valores de teste com score para obter o score de cada registro 
y_scores = df_features.dot(scorecard_scores)

In [33]:
y_scores

,0
0,-79.0
1,16.0
2,222.0
3,189.0
4,223.0
...,...
458114,-32.0
458115,-95.0
458116,107.0
458117,41.0


### Calculando a probabilidade de inadimplência

In [34]:
# divide-se a diferença entre as pontuações e a pontuação mínima por
# a diferença entre a pontuação máxima e a pontuação mínima
# multiplica-se pela diferença entre a soma máxima dos coeficientes e a soma mínima dos coeficientes
# adiciona-se a soma mínima dos coeficientes
sum_coef_score = ((y_scores - min_score) / (max_score - min_score)) * (max_sum_coef - min_sum_coef) + min_sum_coef

In [35]:
y_proba_score = np.exp(sum_coef_score) / (np.exp(sum_coef_score) + 1)

y_proba_score.head()

,0
0,0.136354
1,0.396971
2,0.935714
3,0.898621
4,0.936612


### Definindo pontos de corte

Agora as métricas calculadas serão utilizadas para estipular um ponto de corte e que ajude a decidir se um empréstimo será concedido ou não.

In [36]:
y_pred_proba = modelo_pd.predict_proba(X_test)[:, 1]

taxa_falso_pos, taxa_verdadeiro_pos, thresholds = metrics.roc_curve(y_test, y_pred_proba)

In [38]:
# concatena-se as três colunas
df_cutoffs = pd.concat([pd.DataFrame(thresholds), pd.DataFrame(taxa_falso_pos), pd.DataFrame(taxa_verdadeiro_pos)], axis = 1)

df_cutoffs.columns = ['thresholds', 'taxa_falso_pos', 'taxa_verdadeiro_pos']

df_cutoffs.head()

,thresholds,taxa_falso_pos,taxa_verdadeiro_pos
0,1.999415,0.000000,0.000000
1,0.999415,0.000000,0.000004
2,0.999334,0.000004,0.000004
3,0.998381,0.000004,0.000070
4,0.998328,0.000009,0.000070


In [39]:
df_cutoffs['thresholds'][0] = 1 - 1 / np.power(10, 16)

# calculando score correspondente de cada thresholds
df_cutoffs['score'] = ((np.log(df_cutoffs['thresholds'] / (1 - df_cutoffs['thresholds'])) - min_sum_coef) * ((max_score - min_score) / (max_sum_coef - min_sum_coef)) + min_score).round()
df_cutoffs['score'][0] = max_score
df_cutoffs.head()

,thresholds,taxa_falso_pos,taxa_verdadeiro_pos,score
0,1.000000,0.000000,0.000000,1000.0
1,0.999415,0.000000,0.000004,539.0
2,0.999334,0.000004,0.000004,530.0
3,0.998381,0.000004,0.000070,471.0
4,0.998328,0.000009,0.000070,469.0


In [40]:
def n_aprovado(p):
    return np.where(y_pred_proba >= p, 1, 0).sum()

In [41]:
df_cutoffs['n_aprovado']     = df_cutoffs['thresholds'].apply(n_aprovado)
df_cutoffs['n_rejeitado']    = y_pred_proba.shape[0] - df_cutoffs['n_aprovado']
df_cutoffs['taxa_aprovacao'] = df_cutoffs['n_aprovado'] / y_pred_proba.shape[0]
df_cutoffs['taxa_rejeicao']  = 1 - df_cutoffs['taxa_aprovacao']

In [42]:
df_cutoffs.head()

,thresholds,taxa_falso_pos,taxa_verdadeiro_pos,score,n_aprovado,n_rejeitado,taxa_aprovacao,taxa_rejeicao
0,1.000000,0.000000,0.000000,1000.0,0,458119,0.000000,1.000000
1,0.999415,0.000000,0.000004,539.0,1,458118,0.000002,0.999998
2,0.999334,0.000004,0.000004,530.0,2,458117,0.000004,0.999996
3,0.998381,0.000004,0.000070,471.0,17,458102,0.000037,0.999963
4,0.998328,0.000009,0.000070,469.0,18,458101,0.000039,0.999961


Considerando uma probabilidade de default de 7% com thresholds de 93%, teríamos uma taxa de aprovação de 7.4% e de rejeição de 92.6%

In [43]:
df_cutoffs.iloc[5000: 6200, ]

,thresholds,taxa_falso_pos,taxa_verdadeiro_pos,score,n_aprovado,n_rejeitado,taxa_aprovacao,taxa_rejeicao
5000,0.932937,0.009857,0.138873,219.0,34054,424065,0.074334,0.925666
5001,0.932935,0.009862,0.138873,219.0,34055,424064,0.074337,0.925663
5002,0.932924,0.009862,0.138921,219.0,34066,424053,0.074361,0.925639
5003,0.932923,0.009862,0.138929,219.0,34068,424051,0.074365,0.925635
5004,0.932911,0.009862,0.138942,219.0,34071,424048,0.074372,0.925628
...,...,...,...,...,...,...,...,...
6195,0.923437,0.012314,0.163467,209.0,40248,417871,0.087855,0.912145
6196,0.923435,0.012318,0.163467,209.0,40249,417870,0.087857,0.912143
6197,0.923429,0.012318,0.163489,209.0,40254,417865,0.087868,0.912132
6198,0.923428,0.012323,0.163489,209.0,40255,417864,0.087870,0.912130


O score também pode ser utilizada para selecionar os pontos de corte, um score de 287, default de 3%, teria uma taxa de aprovação de 1.7% e uma taxa de rejeição de 98.3%

In [44]:
df_cutoffs.iloc[700: 900, ]

,thresholds,taxa_falso_pos,taxa_verdadeiro_pos,score,n_aprovado,n_rejeitado,taxa_aprovacao,taxa_rejeicao
700,0.978301,0.001505,0.026228,297.0,6350,451769,0.013861,0.986139
701,0.978300,0.001505,0.026237,297.0,6352,451767,0.013865,0.986135
702,0.978300,0.001505,0.026246,297.0,6354,451765,0.013870,0.986130
703,0.978285,0.001505,0.026276,297.0,6361,451758,0.013885,0.986115
704,0.978284,0.001510,0.026276,297.0,6362,451757,0.013887,0.986113
...,...,...,...,...,...,...,...,...
895,0.974808,0.001898,0.033356,287.0,8072,450047,0.017620,0.982380
896,0.974806,0.001903,0.033356,287.0,8073,450046,0.017622,0.982378
897,0.974774,0.001903,0.033435,287.0,8091,450028,0.017661,0.982339
898,0.974773,0.001907,0.033435,287.0,8092,450027,0.017664,0.982336


In [45]:
# salvando dados
df_scorecard.to_csv('dados/dados_scorecard.csv', index = False)

dados_treino1 = pd.DataFrame(columns = nomes_features, data = X_train)
dados_treino2 = pd.DataFrame(columns = ['target'], data = y_train)
dados_treino  = pd.concat([dados_treino1, dados_treino2], axis = 1)
dados_treino.to_csv('dados/dados_populacao_de_treino.csv', index = False)

---